In [5]:
import pandas as pd
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join('..')))

ENV_NAME = "MOLunarLanderDR-v0" # CHANGE THIS TO THE NAME OF THE ENVIRONMENT
REWARD_DIM = 4 # CHANGE THIS TO THE NUMBER OF OBJECTIVES IN THE ENVIRONMENT
SEEDS = [5,26,47,76,92] # CHANGE THIS TO THE SEEDS YOU USE

from helpers.utils import ENVIRONMENTS_MAP, get_algorithms
ALGORITHMS = get_algorithms(ENV_NAME)

In [6]:
sys.path.append(os.path.abspath(os.path.join('../..')))

from mo_utils.performance_indicators import cardinality
from mo_utils.pareto import filter_pareto_dominated

In [7]:
import warnings
from mo_utils.pareto import filter_pareto_dominated

curr_envs = ENVIRONMENTS_MAP[ENV_NAME]
GENERALIST_FRONT = "eval/discounted_front" # don't change this, we use discounted for evaluations!!
file_path = f"data/{GENERALIST_FRONT}/{ENV_NAME}"
scores_save_path = f"data/cardinality/{ENV_NAME}"

os.makedirs(f"{scores_save_path}", exist_ok=True)

for algo in ALGORITHMS:
    for seed in SEEDS:
        generalist_eums = []
        for env in ENVIRONMENTS_MAP[ENV_NAME]:
            file = f"{file_path}/{algo}/seed_{seed}/{env}.csv"
            assert os.path.exists(file), f"File {file} does not exist"
            data = pd.read_csv(file)
            # Convert dataframe to numpy array of vectors
            data_array = data.to_numpy()
            data_array = filter_pareto_dominated(data_array) # filter again to be sure

            generalist_eums.append(cardinality(data_array))

        data = {f"cardinality/{env}": [generalist_eums[i]] for i, env in enumerate(ENVIRONMENTS_MAP[ENV_NAME])}
        df = pd.DataFrame(data)
        os.makedirs(f"{scores_save_path}/{algo}/", exist_ok=True)
        df.to_csv(f"{scores_save_path}/{algo}/seed_{seed}.csv", index=False)

In [8]:
import numpy as np

scores_save_path = f"data/cardinality/{ENV_NAME}"

table_data = {}

for algo in ALGORITHMS:
    algo_path = f"{scores_save_path}/{algo}/"
    algo_data = []
    
    for seed in SEEDS:
        file = f"{algo_path}/seed_{seed}.csv"
        assert os.path.exists(file), f"File {file} does not exist"
        
        df = pd.read_csv(file)
        algo_data.append(df.values.flatten())
    
    # Convert to numpy array for easier aggregation
    algo_data = np.array(algo_data)  # Shape: (num_seeds, num_envs)
    
    # Calculate mean and error ranges
    mean = np.mean(algo_data, axis=0)  # Mean over seeds
    std_error = np.std(algo_data, axis=0, ddof=1) / np.sqrt(len(SEEDS))  # Standard error
    
    # Format mean ± SE as a string for each environment
    env_results = [
        f"{mean[i]:.2f} ± {std_error[i]:.2f}" for i in range(len(ENVIRONMENTS_MAP[ENV_NAME]))
    ]
    
    # Store in table_data
    table_data[algo] = env_results

env_names = [f"cardinality/{env}" for env in ENVIRONMENTS_MAP[ENV_NAME]]
table_df = pd.DataFrame.from_dict(table_data, orient="index", columns=env_names)

# print the table
print("Mean ± SE Cardinality Table:")
print(table_df.to_string())

# Save the table to a CSV file
results_path = f"{scores_save_path}/combined_table.csv"
table_df.to_csv(results_path, index_label="Algorithm")

Mean ± SE Cardinality Table:
                             cardinality/MOLunarLanderDefault-v0 cardinality/MOLunarLanderHighGravity-v0 cardinality/MOLunarLanderWindy-v0 cardinality/MOLunarLanderTurbulent-v0 cardinality/MOLunarLanderHard-v0 cardinality/MOLunarLanderLowMainEngine-v0 cardinality/MOLunarLanderStartLow-v0 cardinality/MOLunarLanderStartRight-v0
MORL-D(MOSACDiscrete)-SB+PSA                        54.60 ± 3.56                            70.60 ± 2.50                      59.60 ± 3.70                          69.20 ± 2.27                     55.20 ± 2.27                              60.80 ± 2.87                         52.40 ± 3.12                           54.80 ± 3.87
MORL-D(MOSACDiscrete)-SB                            62.60 ± 4.02                            60.40 ± 1.96                      68.00 ± 2.88                          64.40 ± 3.04                     53.00 ± 3.13                              67.40 ± 1.96                         56.60 ± 1.03                           